Exemplos *Classe PerfilILam*
-----------------------
Este notebook apresenta exemplos de utilização da classe *PerfilILam* que representa o perfil metálico do tipo *I laminado*. Estes exemplos tem por objetivo mostrar como utilizar a biblioteca Metalpy, apresentar algumas funcionalidades e servir como validação dos métodos implementados.

Os exemplos apresentados são exemplos já resolvidos obtidos de livros e manuais das seguintes referências:

[1] PFEIL, WALTER. **Estruturas de aço: dimensionamento prático.** 9ª Edição. Rio de Janeiro. LTC, 2022.

[2] AMERICAN INSTITUTE OF STEEL CONSTRUCTION. AISC 360: **Companion to the AISC steel construction manual - Volume 1: Design Examples.** V15.1, Chicago. Illinois, 2015.

-----------------------------------------------------------------------------------------------------------------------------

### Exemplo 1

Este exemplo é o exercícico 6.5.1.a) dos problemas resolvidos do capitulo 6 de [1], o exercicio trata da obtenção do momento fletor resistênte de um perfil I laminado W530X85. Considera-se o perfil contido lateralmente em toda a sua extensão. Aço MR250.

In [1]:
from perfis import PerfilILam
from material import Aco

import pandas as pd

#Definindo o aço do tipo MR250 com as propriedades em kN/cm²
MR250 = Aco(20000, 0.3, 25, 30, 0.6)

#Definindo um perfil I laminado do tipo W com as propriedades em cm
W530X85 = PerfilILam('W530X85', MR250, und='cm', norma='NBR8800')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [4]:
#Respostas do exercicio
r_esb_mesa = 5 # Esbeltez da mesa
r_esb_alma = 46.4 # Esbeltez da alma
r_Zx = 2100 # Módulo plástico
r_Mrdx = 47730 #Momento resistênte

#Respostas com Metalpy
mp_esb_mesa = W530X85.esb_mesa
mp_esb_alma =  W530X85.esb_alma
mp_Zx = W530X85.Zx
# Obtendo o momento fletor resistente em relação ao eixo X(maior inércia)
Mrdx, dados_FLT, dados_FLM, dados_FLA = W530X85.Mrdx(Lb=0, Cb=1, data=True)

columns = 'Referência Metalpy'.split()
index = ['bf/2tf', 'h/tw', 'Zx (cm³)', 'Mrdx(kN.cm)']
data = zip([r_esb_mesa, r_esb_alma, r_Zx, r_Mrdx], [ mp_esb_mesa, mp_esb_alma, mp_Zx, Mrdx])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
bf/2tf,5.0,5.040000
h/tw,46.4,46.300000
Zx (cm³),2100.0,2110.000000
Mrdx(kN.cm),47730.0,47954.545455


In [5]:
# Verificação
tol = 0.02 #tolerência de 2% de erro

assert min(r_Mrdx, Mrdx)/ max(r_Mrdx, Mrdx) >= 1 - tol, 'Valor do momento resistênte excede o tolerência'

-------------------------------------------------------------------------------------------------------------------------
Exemplo 2
---------------
Exemplo E.1D de [3]. Determinação da força axial de compressão resistênte de uma coluna formada por um perfil W19X90 (equivalente ao W360X134 em mm e kg/m) através dos critérios de cálculo da AISC360-16. A coluna é constituida de aço ASTM A992 e comprimento 15ft (457.2 cm). As parâmetros geométricos do perfil estão apresentados abaixo:

* Ag = 26.5 in²
* rx = 6.14 in
* ry = 3.70 in
* bf/2tf = 10.2
* h/tw = 25.9

*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [6]:
from perfis import PerfilI
from normas import AISC360
from material import Aco

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A992 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil W360X134 em unidades de cm
W360X134 = PerfilILam('W360X134', A992, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [18]:
#Respostas do exercicio
r_A = 26.5 # Área total
r_esb_mesa = 10.2 # Esbeltez da mesa
r_esb_alma =  25.9 # Esbeltez da alma
r_ry = 3.7 # Raio de giração em Y
r_rx = 6.14 # Raio de giração em X
r_Fe = 83.3 # Tensão critica de flambagem
r_Fcr = 38.9 # Tensão crítica
r_Ncrd = 927 # Força axial resistênte de cálculo pela AISC

#Respostas com Metalpy
mp_A =  W360X134.A * cm_to_in ** 2
mp_esb_mesa = W360X134.esb_mesa
mp_esb_alma = W360X134.esb_alma
mp_ry = W360X134.ry * cm_to_in
mp_rx = W360X134.rx * cm_to_in


# Calculando a força axial resistênte de cálculo através do método Ncrd e obtendo os dados do cálculo
Lx = 30 * ft_to_cm
Ly = 15 * ft_to_cm
Lz = 1
Ncrd, dados = W360X134.Ncrd(Lx, Ly, 1, data=True)
mp_Ncrd = Ncrd * kN_to_kips
mp_Fe = dados.Fe * kN_cm2_to_ksi
mp_Fcr = dados.Fcr * kN_cm2_to_ksi

columns = 'Referência Metalpy'.split()
index = ['A (in²)', 'bf/2tf', 'h/tw', 'ry (in)', 'rx (in)', 'Fe (ksi)', 'Fcr (ksi)', 'Ncrd (kips)']
data = zip([r_A, r_esb_mesa, r_esb_alma, r_ry, r_rx, r_Fe, r_Fcr, r_Ncrd], [ mp_A, mp_esb_mesa, mp_esb_alma, mp_ry, mp_rx, mp_Fe, mp_Fcr, mp_Ncrd])

df = pd.DataFrame(data,index=index,columns=columns)
df

,Referência,Metalpy
A (in²),26.50,26.545356
bf/2tf,10.20,10.200000
h/tw,25.90,25.900000
ry (in),3.70,3.702428
rx (in),6.14,6.145325
Fe (ksi),83.30,83.276512
Fcr (ksi),38.90,38.889389
Ncrd (kips),927.00,928.462721


In [13]:
# Verificação
tol = 0.01 #tolerência de 1% de erro

assert min(r_Ncrd, mp_Ncrd)/ max(r_Ncrd, mp_Ncrd) >= 1 - tol, 'Valor da força axial resistênte excede a tolerância'

-------------------------------------------------------------------------------------------------------------------------
Exemplo 3
---------------
Exemplo F.1 de [3]. Determinação do momento fletor resistênte de cálculo de uma viga formada por um perfil W18X50 (equivalente ao W460X74 em mm e kg/m) através dos critérios de cálculo da AISC360-16. A coluna é constituida de aço ASTM A992 e comprimento 15ft (457.2 cm). Considera-se três casos de ctravamento lateral:

* 1A - Viga contida em todo seu comprimento (Lb=0)
* 2A - Viga travada a cada 1/3 do seu comprimento (Lb=11.67 in (355.6 cm))
* 3A - Viga travada no centro (Lb=17.5 in (533.4 cm))

*Obs: O exemplo será resolvido em unidades de comprimento em **cm** e de força em **kN**, e os resultados serão convertidos posteriormente para unidades de **in** e **kips***

In [17]:
from perfis import PerfilI
from normas import AISC360
from material import Aco

#Unidades para conversão
in_to_cm = 2.54
cm_to_in = 0.394
ft_to_cm = 30.48
kN_cm2_to_ksi = 1.4504
ksi_to_kN_cm2 = 1 / 1.4504
ft_to_cm = 30.48
kN_to_kips = 0.225

#Definição do Aço com as propriedades em ksi sendo convertidas para kN/cm²
E = 29000 * ksi_to_kN_cm2 
fy = 50 * ksi_to_kN_cm2
fu = 65 * ksi_to_kN_cm2
v = 0.3

#Definindo um aço ASTM A992 em unidades de kN/cm²
A992 = Aco(E, v, fy, fu, 0.6)

#Definindo um perfil W460X74
W460X74 = PerfilILam('W460X74', A992, und='cm', norma='AISC360')

Comparação entre os valores de referência e valores obtidos com Metalpy

In [ ]:
#Respostas do exercicio
r_A = 26.5 # Área total
r_esb_mesa = 10.2 # Esbeltez da mesa
r_esb_alma =  25.9 # Esbeltez da alma
r_ry = 3.7 # Raio de giração em Y
r_rx = 6.14 # Raio de giração em X
r_Fe = 83.3 # Tensão critica de flambagem
r_Fcr = 38.9 # Tensão crítica
r_Ncrd = 927 # Força axial resistênte de cálculo pela AISC

#Respostas com Metalpy
mp_A =  W360X134.A * cm_to_in ** 2
mp_esb_mesa = W360X134.esb_mesa
mp_esb_alma = W360X134.esb_alma
mp_ry = W360X134.ry * cm_to_in
mp_rx = W360X134.rx * cm_to_in


# Calculando a força axial resistênte de cálculo através do método Ncrd e obtendo os dados do cálculo
Lx = 30 * ft_to_cm
Ly = 15 * ft_to_cm
Lz = 1
Ncrd, dados = W360X134.Ncrd(Lx, Ly, 1, data=True)
mp_Ncrd = Ncrd * kN_to_kips
mp_Fe = dados.Fe * kN_cm2_to_ksi
mp_Fcr = dados.Fcr * kN_cm2_to_ksi

columns = 'Referência Metalpy'.split()
index = ['A (in²)', 'bf/2tf', 'h/tw', 'ry (in)', 'rx (in)', 'Fe (ksi)', 'Fcr (ksi)', 'Ncrd (kips)']
data = zip([r_A, r_esb_mesa, r_esb_alma, r_ry, r_rx, r_Fe, r_Fcr, r_Ncrd], [ mp_A, mp_esb_mesa, mp_esb_alma, mp_ry, mp_rx, mp_Fe, mp_Fcr, mp_Ncrd])

df = pd.DataFrame(data,index=index,columns=columns)
df